In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import csv
from scipy.io import arff
from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim


In [138]:
#data = arff.loadarff('universityPolicyTrainingData.arff')
permit = pd.read_csv('DataSet2RulesPermitted.csv')
denied = pd.read_csv('DataSet2RulesDenied.csv')
df = pd.DataFrame(permit)
#print(denied)
#refine = pd.read_csv('step1ExtractedRulesfalseNegatives.csv')
#df = pd.DataFrame(refine)
act = pd.concat([permit,denied])
#print(data)

df= df.drop(columns ='Unnamed: 9')

print()
print(df.describe())
df.head()


       ischair    crs odepartment action position       type udepartment  \
count     8000   8000        8000   8000     8000       8000        8000   
unique       3      5           5      5        4          4           5   
top       true  cs602  admissions   read  faculty  gradebook   registrar   
freq      3363   1652        1674   4000     3203       3000        2226   

       crstaught crstaken  
count       8000     8000  
unique         5        5  
top        cs601    cs101  
freq        1666     1648  


,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken
0,false,ee601,cs,read,faculty,roster,registrar,ee101,ee601
1,false,cs101,ee,read,faculty,roster,ce,ee601,cs101
2,true,ee601,cs,read,advisor,transcript,registrar,ee101,cs602
3,true,ee101,cs,read,advisor,transcript,ce,cs101,cs101
4,true,cs101,registrar,read,staff,application,admissions,cs101,ee101


In [139]:
#print(df.describe())

df.isnull().values.any()
df["position"] = df["position"].astype('category')
df["udepartment"] = df["udepartment"].astype('category')
df["crstaken"] = df["crstaken"].astype('category')
df["crstaught"] = df["crstaught"].astype('category')
df["ischair"] = df["ischair"].astype('category')
df["type"] = df["type"].astype('category')
df["crs"] = df["crs"].astype('category')
df["odepartment"] = df["odepartment"].astype('category')
df["action"] = df["action"].astype('category')

act.isnull().values.any()
act["position"] = act["position"].astype('category')
act["udepartment"] = act["udepartment"].astype('category')
act["crstaken"] = act["crstaken"].astype('category')
act["crstaught"] = act["crstaught"].astype('category')
act["ischair"] = act["ischair"].astype('category')
act["type"] = act["type"].astype('category')
act["crs"] = act["crs"].astype('category')
act["odepartment"] = act["odepartment"].astype('category')
act["action"] = act["action"].astype('category')



df.head(30)
print(df.dtypes)

ischair        category
crs            category
odepartment    category
action         category
position       category
type           category
udepartment    category
crstaught      category
crstaken       category
dtype: object


In [140]:
# Attributes Frequencies
datafreq = pd.DataFrame()
for col in df.columns:
    datafreq=df[col].value_counts()
    #print (datafreq)
    
#print(datafreq)



In [141]:
# Correlation Function

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Writing Data Correlation in a CSV
with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
    
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [164]:
data = df
k=10
datasize= len(act.index)
print(datasize)
km = KModes(n_clusters=k, init='Cao', n_init=1, verbose=2)
df_dummy = pd.get_dummies(df)

clusters = km.fit_predict(data)
df_dummy['clusters'] = clusters

# Print the cluster centroids

centroids=km.cluster_centroids_

print(centroids)

16000
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 33461.0
[['true' 'cs602' 'admissions' 'read' 'faculty' 'application' 'admissions'
  'cs601' 'ee601']
 ['false' 'ee601' 'registrar' 'addscore' 'student' 'gradebook'
  'registrar' 'cs602' 'cs602']
 ['na' 'ee101' 'ce' 'read' 'advisor' 'roster' 'registrar' 'ee101' 'ee101']
 ['na' 'cs101' 'cs' 'changescore' 'faculty' 'gradebook' 'ce' 'cs101'
  'cs101']
 ['true' 'cs601' 'cs' 'read' 'student' 'transcript' 'registrar' 'ee601'
  'cs601']
 ['true' 'cs101' 'ee' 'read' 'faculty' 'roster' 'registrar' 'cs602'
  'cs101']
 ['true' 'ee101' 'admissions' 'readmyscores' 'student' 'gradebook' 'cs'
  'cs101' 'ee101']
 ['false' 'cs601' 'registrar' 'read' 'faculty' 'roster' 'admissions'
  'cs101' 'cs601']
 ['false' 'ee601' 'cs' 'read' 'student' 'application' 'admissions'
  'ee101' 'cs101']
 ['true' 'ee601' 'ce' 'changescore' 'faculty' 'gradebook' 'registrar'
  'ee101' 'ee601']]


In [165]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = data.index.values
cluster_map['cluster'] = km.labels_

#valuescount=pd.DataFrame()
#temp=[]

for i in range(0, k):
    ind=cluster_map.cluster == i
    #print(ind)

In [166]:
cluster_map
ndf=pd.concat([data,cluster_map], axis=1)
ndf=ndf.drop(columns='data_index')
ndf

,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken,cluster
0,false,ee601,cs,read,faculty,roster,registrar,ee101,ee601,8
1,false,cs101,ee,read,faculty,roster,ce,ee601,cs101,5
2,true,ee601,cs,read,advisor,transcript,registrar,ee101,cs602,4
3,true,ee101,cs,read,advisor,transcript,ce,cs101,cs101,3
4,true,cs101,registrar,read,staff,application,admissions,cs101,ee101,0
5,true,cs601,admissions,read,advisor,transcript,registrar,ee601,cs602,4
6,true,ee101,admissions,read,faculty,transcript,cs,cs602,ee601,0
7,false,ee101,registrar,addscore,staff,gradebook,ee,cs101,cs101,1
8,false,cs601,cs,checkstatus,student,application,ee,ee601,cs601,4
9,na,cs601,ee,checkstatus,student,application,ee,cs101,cs101,3


In [167]:
#centroids
ndf.groupby('cluster').count()

,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken
cluster,,,,,,,,,
0,1422,1422,1422,1422,1422,1422,1422,1422,1422
1,979,979,979,979,979,979,979,979,979
2,906,906,906,906,906,906,906,906,906
3,933,933,933,933,933,933,933,933,933
4,886,886,886,886,886,886,886,886,886
5,723,723,723,723,723,723,723,723,723
6,534,534,534,534,534,534,534,534,534
7,501,501,501,501,501,501,501,501,501
8,601,601,601,601,601,601,601,601,601


In [169]:
# Calculating Attributes Effectevness in each cluster
temp = pd.DataFrame()
datafreq = pd.DataFrame()
extract = pd.DataFrame()
rules = pd.DataFrame(columns=['ischair','crs','odepartment','action','position',
                              'type','udepartment','crstaught','crstaken'])
print(datafreq)
clusTemp = pd.DataFrame()
for i in range(k):
    print()
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(clussize)
    for col in df.columns:
        datafreq = act[col].value_counts().to_dict()
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        #temp= pd.DataFrame.from_dict(temp)
        #print(temp)
        #print(datafreq)
        for key, value in datafreq.items():
            #if key not in temp.keys():
                #print (key, 'is not in the cluster')
        #    datafreq["count"] = df.loc[col == attr].size()
            #print(key)
            x = value/datasize
            #print(x)
            for key1, value1 in temp.items():
                
                if key == key1:
                    y = value1/clussize
                    #print(y)
                    
                    if y-x >= 0.350:
                        #print('the effectevniess of ', key, y, x, 'is: ', y-x)
                        #l.append(key)
                        #print(col)
                        print(col,'=',key,',',end="")
                    #if y-x < -0.259:
                         #print (col,'!=',key,',',end="")

  


Empty DataFrame
Columns: []
Index: []

CLUSTER NUMBER 0
action = read ,udepartment = admissions ,
CLUSTER NUMBER 1
action = addscore ,type = gradebook ,
CLUSTER NUMBER 2
action = read ,type = roster ,udepartment = registrar ,
CLUSTER NUMBER 3
action = changescore ,position = faculty ,type = gradebook ,
CLUSTER NUMBER 4
ischair = true ,action = read ,type = transcript ,
CLUSTER NUMBER 5
action = read ,type = roster ,
CLUSTER NUMBER 6
action = readmyscores ,type = gradebook ,crstaken = ee101 ,
CLUSTER NUMBER 7
ischair = false ,crs = cs601 ,action = read ,position = faculty ,type = roster ,crstaken = cs601 ,
CLUSTER NUMBER 8
position = student ,type = application ,udepartment = admissions ,
CLUSTER NUMBER 9
crs = ee601 ,action = changescore ,position = faculty ,type = gradebook ,crstaken = ee601 ,

In [95]:
datacorr = pd.DataFrame()
l=['odepartment','udepartment','crstaught','crstaken','crs']

for i in range(k):
    print()
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    for col in df.columns:
        if col in l:
    #writer = csv.writer(f, dialect='excel')
            for col2 in df.columns:
                if col2 in l:
                    confusion_matrix = pd.crosstab(act[col], act[col2]).as_matrix()
                    datacorr = cramers_v(confusion_matrix)
                    #print('correlation between ',col,'and', col2,'is : ',datacorr)
                    temp=clusTemp[col].value_counts().to_dict()    
                    confusion_matrix = pd.crosstab(clusTemp[col], clusTemp[col2]).as_matrix()
                    temp = cramers_v(confusion_matrix)
                    #print (temp, datacorr)
                    if temp > datacorr :
                        #print (temp, datacorr)
                        c= temp - datacorr
                        if c >= 0.2:
                            #print (temp, datacorr,c)
                            print(col,'==', col2,',' ,end="")
                    else:
                        c= datacorr - temp 
                        if c >= 0.10:
                    
                            #print (temp, datacorr, c)
                            print(col,'!==', col2,',' ,end="")


CLUSTER NUMBER 0


/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



CLUSTER NUMBER 1

CLUSTER NUMBER 2


In [337]:
from sklearn.decomposition import PCA
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
        distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
        return distances
#clusters=km.fit_predict(data2D)
#centroids = km.cluster_centers_
#data2D=PCA(n_components=2).fit_transform(data)
distances = []
for i, (cx, cy) in enumerate(centroids):
    mean_distance = k_mean_distance(data2D, cx, cy, i, clusters)
    distances.append(mean_distance)

print(distances)

ValueError: could not convert string to float: 'true'

In [211]:
# Correlation in Each Cluster

with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
        
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
